# Домашняя работа 1

Аксенова Анна БКЛ182

Импортируем библиотеки и модули, которые понадобятся в процессе

In [311]:
import json
import collections
import re

Считываем текст и кладем его как список словарей в переменную tweets

In [ ]:
tweets = ([json.loads(line) for line in 
           open('hw_3_twitter.json', 'r', encoding='utf-8')])

**Задание 1**

Каждый твит - отдельный словарь, значит, количество твитов - количество словарей в списке

In [ ]:
print('Количество твитов: ', end='')
print(len(tweets))

**Задание 2**

В удаленных твитах есть ключ *'delete'*. Посчитаем такие твиты и разделим их количество на количество всех твитов

In [ ]:
deleted = 0  # Счетчик
for i in range(len(tweets)):  # Проходимся по всем твитам через индекс, тк tweets - список
    if 'delete' in tweets[i].keys():
        deleted += 1
print('Процент удаленных твитов: ', end='')
print(str(deleted / len(tweets) * 100) + '%')

**Задание 3**

За язык твита отвечает ключ *'lang'*, соберем кучу языков, а потом с помощью *Counter* посчитаем сколько раз встречался каждый язык

In [ ]:
languages = []  # Сюда мы будем складывать языки каждого твита
for i in range(len(tweets)):
    for key, value in tweets[i].items():
        if key == 'lang':
            languages.append(value)
lang_rates = collections.Counter(languages).most_common()  # Считаем сколько раз встречается каждый язык 
# и сортируем в порядке убывания частотности
print('Язык'+'\t\t'+'Количество твитов')
for i in range(10):
    print(str(lang_rates[i][0]) + '\t\t' + str(lang_rates[i][1]))


**Задание 4**

У каждого твита есть id пользователя, его создавшего. Алгоритм решения такой же, как в задании 3

In [ ]:
users = []  # Складываем сюда id всех пользователей
for i in range(len(tweets)):
    if 'delete' not in tweets[i].keys():
        users.append(tweets[i]["user"]["id"])
users = collections.Counter(users)
active_users = 0  # Счетчик
for key, value in users.items():
    if value > 1: 
        active_users += 1
print('Количество пользователей, имеющих более одного твита:',
      active_users)

**Задание 5**

Хэштэги лежат в *'entities'* в значениях ключа *'hashtags'*. При этом хэштэги - список словарей, который тоже нужно распаковать. Для подсчетов самых популярных хэштегов пользуемся все тем же алгоритмом.

In [ ]:
hashtags = []  # Складываем сюда все хэштеги
for i in range(len(tweets)):
    if 'delete' not in tweets[i].keys():
        hash_list = tweets[i]["entities"]["hashtags"]
        hashtags.extend([hash_list[i]["text"] for i in range(len(hash_list))])
hashtags = collections.Counter(hashtags).most_common()
print('Топ самых популярных хэштегов')
for i in range(20):
    print(str(i + 1) + '. ' + str(hashtags[i][0]))

**Задание 6** 

*(при поддержке Кати Такташевой)*

Найдем самые частотные слова. Текст твита лежит в значении ключа *'text'*, его нужно очистить от пунктуации, ссылок, хэштегов и хэндлов.

In [307]:
# Переменные, которые понадобятся для очистки текста:
punctuation = (list(""".,…;:!”?$%^&*№()_—=+|[]{}\/"<>`~±§«»°1234567890""") 
               + ['- ', ' -', ' - ', " '", "' "]) # не удаляем "-", потому что встречается в композитах
links = r'http.+?($|\s)'  # регулярное выражение для удаления ссылок
handles = r'@.*?($|\s)'  # регулярное выражение для удаления хэндлов
hashs = r'#.*?($|\s)'  # регулярное выражение для удаления хэштегов

In [312]:
parced = []  # сюда мы складываем слова
for i in range(len(tweets)):
    if (("retweeted_status" not in tweets[i].keys()) 
        and ('delete' not in tweets[i].keys()) and (tweets[i]['lang'] == 'en')):
        text = tweets[i]["text"]
        for char in punctuation:
            text = text.strip().replace(char, '')
        text = re.sub(links, r'', text)  # убираем ссылки
        text = re.sub(handles, r'', text)  # убираем хэндлы
        text = re.sub(hashs, r'', text)  # убираем хэштеги
        text = re.sub(r'[^a-zA-Z\'\s-]', r'', text) # убираем смайлики и т.п.
        text = text.lower()
        list_of_words = text.split()
        parced.extend(list_of_words)
common_words = collections.Counter(parced).most_common()
print('Рейтинг частотности слов с количеством вхождений')
for i in range(10):
    print((str(i + 1) + '. ' + str(common_words[i][0]) 
           + '\t\t' + str(common_words[i][1])))

Рейтинг частотности слов с количеством вхождений
1. the		126
2. to		86
3. a		78
4. i		74
5. and		64
6. is		51
7. you		48
8. of		46
9. for		42
10. it		42


**Задание 7**

Найдем количество фолловеров для каждого пользователя (*'user'*). Для этого создадим словарь с ключами - именами пользователей (*'name'*) и значениями - количеством фолловеров (*'follower_count'*). Затем отсортируем словарь по значениям.

In [ ]:
popular_users = {}  # Cловарь для пользователей
for i in range(len(tweets)):
    if 'delete' not in tweets[i].keys():
        popular_users[tweets[i]['user']['name']] = tweets[i]['user']['followers_count']
count = 0  # Счетчик для печати
print('Имя пользователя', '\t', 'Количество фолловеров')
for user in sorted(popular_users, key=popular_users.get, reverse=True):
    print(user, '\t\t', popular_users[user])
    count += 1
    if count == 10:
        break

**Задание 8**

Используемые приложения лежат в значениях ключа *'source'*. При этом упоминания приложений имеют вид *>**Application**</a*.

In [ ]:
apps = [] # Сюда складываем все приложения для твитов
for i in range(len(tweets)):
    if 'delete' not in tweets[i].keys():
        source = tweets[i]['source']
        source = re.search(r'>(.+?)</a>', source)
        apps.append(source.group(1))
apps = collections.Counter(apps).most_common()
print('Топ самых популярных приложений для твитов')
for i in range(10):
    print(str(i + 1) + '. ' + apps[i][0])